In [ ]:
%pip install langchain langchain_openai openai --upgrade

In [ ]:
import os 

# Update your API key here:
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [1]:
from openai import OpenAI

client = OpenAI()

speech_file_path = "speech.mp3"
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Today is a wonderful day to build something people love!",
)

response.stream_to_file(speech_file_path)

/var/folders/_y/20jl658s4jl0zvy5c0x0c5140000gn/T/ipykernel_41345/3752778934.py:12: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [3]:
# Let's automate a BBC News Article and Generate The Audio for It:
from langchain_community.document_loaders import WebBaseLoader

url = 'https://www.bbc.co.uk/news/uk-politics-68889342'

loader = WebBaseLoader("https://www.bbc.co.uk/news/uk-politics-68889342")

# Extract the text:
data = loader.load()

In [8]:
# Joining all of the documents and removing any redundant characters:
web_text = " ".join([doc.page_content for doc in data]).replace('\n', '')

In [9]:
len(web_text) # Checking the character length.

7768

In [12]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

# Map
map_template = """The following is a set of documents for a BBC News Article:
{text}
Based on this list of docs, please extract the story and make it sound engaging:
"""
map_prompt = ChatPromptTemplate.from_template(map_template)

# If over 4090 characters, then summarize:
if len(web_text) > 4090:
    chain = load_summarize_chain(ChatOpenAI(), 
                                 chain_type="map_reduce", 
                                 map_prompt=map_prompt, 
                                 combine_prompt=map_prompt)
    
    result = chain.invoke({"input_documents": [Document(page_content=web_text)]})
    web_text = result['output_text']

# TODO - You might also want to clean up the text anyway to ensure an effective story is generated through the 
# text to speech

In [13]:
print(web_text)

Labour Deputy Leader Angela Rayner has accused the government of reneging on its promise to ban no-fault evictions in England, claiming they have "caved into vested interests." Despite her pressing questions to Deputy PM Oliver Dowden, the government has failed to provide a timeline for the implementation of the ban, sparking criticism from Rayner.

The Housing Secretary has conceded that the ban may not be in place by the next general election, as the Renters (Reform) Bill faces scrutiny in the House of Lords. This bill, which aims to restrict landlords' ability to evict tenants without a valid reason, has faced delays and amendments due to concerns raised by Conservative MPs about potential burdens on landlords.

In a heated exchange at Prime Minister's Questions, Rayner challenged Dowden to prioritize the ban on no-fault evictions, highlighting the risk of homelessness for nearly a million families. She also lambasted the government's efforts to ban leaseholds, arguing it would not 

In [14]:
speech_file_path = "speech.mp3"
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=web_text,
)

response.stream_to_file(speech_file_path)

/var/folders/_y/20jl658s4jl0zvy5c0x0c5140000gn/T/ipykernel_41345/453047679.py:8: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)
